In [ ]:
#doing all again...cry emojisss
print("okay report")

In [ ]:
%pwd

In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [ ]:
#extracted_data=load_pdf_file(data='Data/')

In [ ]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
#from sentence_transformers import SentenceTransformer
#model = SentenceTransformer("all-MiniLM-L6-v2")
#embeddings = model.encode(["Hello world", "How are you?"])
#print(embeddings)

In [ ]:
#embeddings = download_hugging_face_embeddings()

In [ ]:
#query_result = embeddings.embed_query("Hello world")
#print("Length", len(query_result))

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
#HUGGINGFACEHUB_API_TOKEN=os.environ.get('HUGGINGFACEHUB_API_TOKEN')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("What is Acne?")

In [ ]:

retrieved_docs

In [ ]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
#now i don't have the acess to openAI api key as its paid now ...so trying for hugging face openAI

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

In [ ]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Example question
query = "What is acromegaly and gigantism?"
response = pipe(query, max_new_tokens=100, temperature=0.4)
print(response[0]["generated_text"])

In [ ]:
!pip uninstall transformers -y

In [ ]:
!pip install transformers==4.41.1

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

response = pipe("What is acromegaly and gigantism?", max_new_tokens=100)
print(response[0]["generated_text"])

def ask_question(question):
    response = pipe(question, max_new_tokens=100)
    return response[0]["generated_text"]

print(ask_question("What is diabetes?"))

In [ ]:
def ask_question(question):
    response = pipe(question, max_new_tokens=100)
    return response[0]["generated_text"]

print(ask_question("What is diabetes?"))

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.language_models import LanguageModelLike

class LocalLLMWrapper(LanguageModelLike):
    def __init__(self, pipe):
        self.pipe = pipe

    def invoke(self, input, **kwargs):
        question = input["input"] if isinstance(input, dict) else input
        result = self.pipe(question, max_new_tokens=200)
        return {"answer": result[0]["generated_text"]}

local_llm = LocalLLMWrapper(pipe)

# Then use this in your LangChain chain
question_answer_chain = create_stuff_documents_chain(local_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is hypertension?"})
print(response["answer"])

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.language_models import LanguageModelLike

# Load your local model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Wrap into LangChain-compatible class
class LocalLLMWrapper(LanguageModelLike):
    def __init__(self, pipe):
        self.pipe = pipe

    def invoke(self, input, **kwargs):
        question = input["input"] if isinstance(input, dict) else input
        result = self.pipe(question, max_new_tokens=200)
        return {"answer": result[0]["generated_text"]}

local_llm = LocalLLMWrapper(pipe)

# ✅ Define prompt (this was missing before)
system_prompt = (
    "You are a helpful medical assistant. "
    "Use the context below to answer the user's question. "
    "If the answer is not in the context, say 'I don't know.'\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

# ✅ Connect LangChain components
question_answer_chain = create_stuff_documents_chain(local_llm, prompt)

# Assuming you already have `retriever` defined from Pinecone
# Example: retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# 🔍 Ask a question
response = rag_chain.invoke({"input": "What is hypertension?"})
print("Answer:", response["answer"])

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec

# 1. Load API key from .env (or paste directly)
import os
from dotenv import load_dotenv
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

# 2. Connect to your existing Pinecone index
index_name = "medicalbot"  # use your actual index name

# 3. Load same embedding model used to create index
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 4. Load the vector store from Pinecone
vectorstore = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding_model
)

# 5. Now create the retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
%pwd


In [ ]:
import os
os.chdir("../")

In [ ]:
%pwd

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [ ]:
extracted_data=load_pdf_file(data='Data/')

In [ ]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings